In [1]:
!pip install torch torchvision torchaudio torch-geometric pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import pandas as pd
df = pd.read_csv('Tic tac initial results.csv')
df.head()

,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,?,?,?,loss
1,4,7,2,6,?,?,?,win
2,0,8,1,6,5,?,?,draw
3,4,7,2,3,?,?,?,draw
4,0,4,2,1,?,?,?,win


In [5]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder



label_encoder = LabelEncoder()
df['CLASS'] = label_encoder.fit_transform(df['CLASS'])


def create_tic_tac_toe_graph(moves, outcome):
    board_size = 9
    node_features = np.zeros((board_size, 1))


    for i, move in enumerate(moves):
        if move != '?':
            node_features[int(move)] = 1

    node_features = torch.tensor(node_features, dtype=torch.float)


    edge_index = []
    for i in range(len(moves) - 1):
        if moves[i] != '?' and moves[i + 1] != '?':
            edge_index.append([int(moves[i]), int(moves[i + 1])])


    if edge_index:
      edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
      edge_index = torch.tensor([], dtype=torch.long).reshape(0, 2)

    #  outcome to tensor
    outcome = torch.tensor(outcome, dtype=torch.long)

    return Data(x=node_features, edge_index=edge_index, y=outcome)

# dataset to graph data
graph_data_list = []
for _, row in df.iterrows():
    if 'MOVE8' in df.columns and 'MOVE9' in df.columns:
        moves = row[['MOVE1', 'MOVE2', 'MOVE3', 'MOVE4', 'MOVE5', 'MOVE6', 'MOVE7','MOVE8', 'MOVE9']].tolist()
    else:
        # Handle the case where 'MOVE8' and 'MOVE9' are not present
        moves = row[['MOVE1', 'MOVE2', 'MOVE3', 'MOVE4', 'MOVE5', 'MOVE6', 'MOVE7']].tolist()
        # Add placeholder values for missing moves
        moves.extend(['?', '?'])
    #  outcome to tensor
    outcome =  row['CLASS']
    graph_data = create_tic_tac_toe_graph(moves, outcome)
    graph_data_list.append(graph_data)


In [6]:
import pandas as pd
df2 = pd.read_csv('Tic tac initial results.csv')
df2.head()

,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,?,?,?,loss
1,4,7,2,6,?,?,?,win
2,0,8,1,6,5,?,?,draw
3,4,7,2,3,?,?,?,draw
4,0,4,2,1,?,?,?,win


In [1]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc1 = torch.nn.Linear(16, 16)  # First fully connected layer
        self.fc2 = torch.nn.Linear(16, 9) # Output 9 scores for 9 moves Output layer)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Handle empty edge indices
        if edge_index.numel() == 0:
            x = torch.zeros_like(x)  # Set node features to zero if no edges
        else:
            x = F.relu(self.conv1(x, edge_index))
            x = F.relu(self.conv2(x, edge_index))

        x = torch.mean(x, dim=0, keepdim=True)

        # Ensure the shape is [1, 16] before feeding to fully connected layers
        if x.shape[1] != 16:
            x = torch.zeros(1, 16)  # If the shape isn't right, set it to zeros with the correct shape

        print("Reshaped x shape:", x.shape)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


ModuleNotFoundError: No module named 'torch_geometric'

In [8]:
from torch_geometric.loader import DataLoader

# Split data into train and test sets
train_size = int(0.8 * len(graph_data_list))
train_data = graph_data_list[:train_size]
test_data = graph_data_list[train_size:]

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)


model = GNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        target = data.y.long()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print(f'Train Loss: {loss.item()}')


def evaluate():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            out = model(data)
            _, predicted = torch.max(out, 1)
            total += data.y.size(0)
            correct += (predicted == data.y.view(-1)).sum().item()
        print(f'Accuracy: {correct / total * 100:.2f}%')


train()
evaluate()


Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.924926519393921
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.8585550785064697
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.7860691547393799
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.7193388938903809
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.6339985132217407
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.5154013633728027
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.4853262901306152
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.4403929710388184
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.3721762895584106
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.3395519256591797
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.6541481018066406
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.1882461309432983
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.668696880340576
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.1144945621490479
Reshaped x shape: torch.Size([1, 16]

In [9]:
torch.save(model.state_dict(), 'gnn_model.pth')

print("Model saved as gnn_model.pth")

Model saved as gnn_model.pth


In [10]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [11]:
class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc1 = torch.nn.Linear(16, 16)
        self.fc2 = torch.nn.Linear(16, 9)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = torch.mean(x, dim=0)
        x = F.relu(self.fc1(x))   # Pass through first fully connected layer
        x = self.fc2(x)           # Pass through second fully connected layer
        return x

In [12]:
class TicTacToe:
    def __init__(self):
        self.reset_game()
        self.model = self.load_model()  # Load your trained GNN model

    def reset_game(self):
        self.board = np.full((3, 3), ':p')
        self.current_player = 'X'
        self.game_over = False
        self.winner = None
        self.winning_cells = []  # Track winning cells
        print("Game reset.")

    def load_model(self):
        try:

            model = GNNModel()
            model.load_state_dict(torch.load('gnn_model.pth'))
            model.eval()
            print("\n")
            return model
        except FileNotFoundError:
            print("Error: The model file 'gnn_model.pth' was not found.")
        except Exception as e:
            print(f"Error loading the model: {e}")

    def suggest_move(self, max_attempts=10):
        if not self.game_over and self.current_player == 'O':  # AI's turn
            attempts = 0
            while attempts < max_attempts:
                try:
                    graph_data = self.create_graph_data(self.board)
                    with torch.no_grad():
                        output = self.model(graph_data)

                        # model output to numpy array
                        output = output.squeeze().numpy()
                        print(f"Model output: {output}")

                        #  (already occupied positions)
                        valid_moves_mask = (self.board.flatten() == ':p')
                        output[~valid_moves_mask] = -np.inf

                        # Find the index of the highest score among valid moves
                        move_index = np.argmax(output)
                        move = divmod(move_index, 3)

                        # Validate the suggested move
                        if self.is_valid_move(move):
                            self.make_move(*move)
                            print(f"AI makes move at position: {move}")
                            return  # Exit after making a valid move
                        else:
                            attempts += 1
                            print(f"Suggested move {move} is invalid or already taken.")
                except Exception as e:
                    print(f"Error during move suggestion: {e}")
                    attempts += 1

            #  notify the user
            print("Max attempts reached. No valid moves found.")

    def is_valid_move(self, move):
        x, y = move
        return 0 <= x < 3 and 0 <= y < 3 and self.board[x, y] == ':p'

    def create_graph_data(self, board):
        try:
            board = np.where(board == 'X', 1, np.where(board == 'O', -1, 0))
            node_features = torch.tensor(board, dtype=torch.float).view(-1, 1)

            edge_index = []
            for i in range(3):
                for j in range(3):
                    if i < 2:
                        edge_index.append([i * 3 + j, (i + 1) * 3 + j])
                    if j < 2:
                        edge_index.append([i * 3 + j, i * 3 + (j + 1)])

            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
            print("Graph data created successfully.")
            return Data(x=node_features, edge_index=edge_index)
        except Exception as e:
            print(f"Error creating graph data: {e}")
            raise

    def make_move(self, x, y):
        if not self.game_over and self.board[x, y] == ':p':
            self.board[x, y] = self.current_player
            print(f"Player {self.current_player} makes a move at ({x}, {y})")
            if self.check_winner():
                self.game_over = True
                self.winner = self.current_player
                print(f"Player {self.winner} wins!")
            elif np.all(self.board != ':p'):
                self.game_over = True
                self.winner = 'Draw'
                print("The game is a draw!")
            else:
                self.current_player = 'O' if self.current_player == 'X' else 'X'
                self.suggest_move()
            self.update_ui()

    def check_winner(self):
        for i in range(3):
            if np.all(self.board[i, :] == self.current_player) or np.all(self.board[:, i] == self.current_player):
                self.winning_cells = [(i, j) for j in range(3)] if np.all(self.board[i, :] == self.current_player) else [(j, i) for j in range(3)]
                return True
        if self.board[0, 0] == self.current_player and self.board[1, 1] == self.current_player and self.board[2, 2] == self.current_player:
            self.winning_cells = [(i, i) for i in range(3)]
            return True
        if self.board[0, 2] == self.current_player and self.board[1, 1] == self.current_player and self.board[2, 0] == self.current_player:
            self.winning_cells = [(i, 2 - i) for i in range(3)]
            return True
        return False

    def update_ui(self):
        for i in range(3):
            for j in range(3):
                button = self.buttons[i][j]
                button.description = self.board[i, j]
                button.disabled = self.board[i, j] != ':p'

                button.style.button_color = 'black'

        if self.game_over:
            if self.winner == 'Draw':
                result_text = 'The game is a draw!'
            else:
                result_text = f'Player {self.winner} wins!'

                for (x, y) in self.winning_cells:
                    button = self.buttons[x][y]
                    button.style.button_color = 'pink'
            self.result_label.value = result_text
            print("UI updated.")

    def create_ui(self):
        self.buttons = [[None for _ in range(3)] for _ in range(3)]
        button_grid = widgets.GridBox(
            children=[widgets.Button(description=':p', layout=widgets.Layout(width='100px', height='100px')) for _ in range(9)],
            layout=widgets.Layout(grid_template_columns='repeat(3, 100px)')
        )
        self.result_label = widgets.Label(value='')
        self.ui = widgets.VBox([button_grid, self.result_label])
        for i in range(3):
            for j in range(3):
                button = button_grid.children[i * 3 + j]
                button.on_click(lambda btn, x=i, y=j: self.make_move(x, y))
                self.buttons[i][j] = button
        self.update_ui()
        print("\n")
        print("\n             TIC - TAC - TOE")
        return self.ui


In [13]:
game = TicTacToe()
display(game.create_ui())

Game reset.





             TIC - TAC - TOE


Player X makes a move at (0, 0)
Graph data created successfully.
Model output: [ 1.0415757   0.5836879   0.27025282 -7.033459   -7.2560387  -6.98179
 -6.8390226  -7.310323   -7.7287707 ]
Player O makes a move at (0, 1)
AI makes move at position: (0, 1)
Player X makes a move at (1, 1)
Graph data created successfully.
Model output: [ 0.43504426  0.6397007   0.5384697  -6.7331805  -6.844539   -6.6216927
 -6.4827566  -6.9215693  -7.27015   ]
Player O makes a move at (0, 2)
AI makes move at position: (0, 2)
Player X makes a move at (2, 2)
Player X wins!
UI updated.
